In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
load_dotenv(override=True)
engine = create_engine(os.environ.get("ALCHEMY_DATABASE_URL"), connect_args={'connect_timeout': 600})

## Query Function


In [2]:
#function definition for running queries
def run_query(query):
    with engine.connect() as connection:
        return pd.read_sql(query, connection)

## Roll Up


In [3]:
# Roll Up
roll_up_query = """
SELECT D.year, COUNT(G.id) AS total_games 
FROM game_fact_table AS G 
JOIN dimDate D ON G.releaseDate = D.date 
GROUP BY D.year;
"""
roll_up_df = run_query(roll_up_query)
print("Roll Up Query Result")
print(roll_up_df)

Roll Up Query Result
    year  total_games
0   1997            2
1   1998            1
2   1999            3
3   2000            2
4   2001            4
5   2002            1
6   2003            3
7   2004            7
8   2005            7
9   2006           69
10  2007          109
11  2008          176
12  2009          343
13  2010          309
14  2011          286
15  2012          357
16  2013          492
17  2014         1634
18  2015         2693
19  2016         4416
20  2017         6334
21  2018         8183
22  2019         7827
23  2020         9647
24  2021        12392
25  2022        13989
26  2023        15542
27  2024        12580
28  2025            2


## Dice


In [9]:
# Dice
dice_query = """
SELECT name, price, metacriticScore 
FROM game_fact_table 
WHERE price > 20 AND metacriticScore > 80;
"""
dice_df = run_query(dice_query)
print("Dice Query Result")
print(dice_df)

Dice Query Result
                                       name  price  metacriticScore
0          Medieval II: Total War™ Kingdoms  24.99               85
1                         BioShock Infinite  29.99               94
2               Sid Meier's Civilization® V  29.99               90
3              Borderlands Game of the Year  29.99               81
4    Total War: EMPIRE – Definitive Edition  24.99               90
..                                      ...    ...              ...
235                  Dungeons of Hinterberg  26.99               82
236                           Blasphemous 2  29.99               83
237                        Magical Delicacy  24.99               84
238                                  Ultros  22.49               83
239       Kunitsu-Gami: Path of the Goddess  49.99               84

[240 rows x 3 columns]


## Drill Down


In [7]:
# Drill Down

drill_down_query = """
SELECT g.name AS game, g.price, de.name AS developer
FROM Game_Fact_Table AS g
LEFT JOIN dimDeveloper dd ON dd.dimDeveloperId = g.dimDeveloperId
LEFT JOIN developer de ON de.groupId = dd.dimDeveloperId
WHERE de.name = 'Valve' AND G.reviews !='none'
ORDER BY game;
"""

drill_down_df = run_query(drill_down_query)
print("Drill Down Query Result")
print(drill_down_df)

Drill Down Query Result
                                game  price developer
0                        Alien Swarm   0.00     Valve
1                  Aperture Desk Job   0.00     Valve
2     Are you ready for Valve Index?   0.00     Valve
3                           Artifact   0.00     Valve
4                     Counter-Strike   9.99     Valve
5     Counter-Strike: Condition Zero   9.99     Valve
6     Counter-Strike: Condition Zero   9.99     Valve
7   Counter-Strike: Global Offensive   0.00     Valve
8             Counter-Strike: Source   9.99     Valve
9                      Day of Defeat   4.99     Valve
10             Day of Defeat: Source   9.99     Valve
11                Deathmatch Classic   4.99     Valve
12                            Dota 2   0.00     Valve
13                   Dota Underlords   0.00     Valve
14                         Half-Life   9.99     Valve
15                       Half-Life 2   9.99     Valve
16           Half-Life 2: Deathmatch   4.99     Valve
17  

In [5]:
# Drill Down

drill_down_query = """
SELECT g.name AS game, g.price, pu.name AS publisher
FROM Game_Fact_Table AS g
LEFT JOIN dimPublisher dp ON dp.dimPublisherId = g.dimpublisherId
LEFT JOIN publisher pu ON pu.groupId = dp.dimPublisherId
WHERE pu.name = 'Ubisoft'
ORDER BY game;
"""

drill_down_df_2 = run_query(drill_down_query)
print("Drill Down Query Result")
print(drill_down_df_2)

Drill Down Query Result
                              game  price publisher
0    Dark Messiah of Might & Magic   9.99   Ubisoft
1    Dark Messiah of Might & Magic   9.99   Ubisoft
2                 Bloody Good Time   4.99   Ubisoft
3        Trackmania United Forever  29.99   Ubisoft
4       TrackMania Nations Forever   0.00   Ubisoft
..                             ...    ...       ...
151      Anno 1602 History Edition   4.99   Ubisoft
152      Anno 1701 History Edition   4.99   Ubisoft
153                     Rocksmith+   0.00   Ubisoft
154  Avatar: Frontiers of Pandora™  41.99   Ubisoft
155                Myst III: Exile  14.99   Ubisoft

[156 rows x 3 columns]


## Slice


In [8]:
# Slice
slice_query = """
SELECT name AS game, releaseDate, price 
FROM game_fact_table 
WHERE releaseDate >= '2022-01-01';
"""
slice_df = run_query(slice_query)
print("Slice Query Result")
print(slice_df)

Slice Query Result
                               game releaseDate  price
0      Borderlands Game of the Year  2023-08-31  29.99
1           Train Simulator Classic  2022-04-21  24.99
2           Train Simulator Classic  2022-04-21  24.99
3           Train Simulator Classic  2022-04-21  24.99
4           Train Simulator Classic  2022-04-21  24.99
...                             ...         ...    ...
42108        Storm Cleaner Playtest  2024-08-27   0.00
42109  Spaceborne Fighters Playtest  2024-08-31   0.00
42110            NET.CRAWL Playtest  2024-08-30   0.00
42111    Food Truck Empire Playtest  2024-08-30   0.00
42112               Carbon Playtest  2024-09-01   0.00

[42113 rows x 3 columns]


In [4]:
# Slice
slice_query = """
SELECT G.name AS game, G.releaseDate, G.price
FROM Game_Fact_Table AS G
WHERE YEAR(G.releaseDate) = 2023;
"""
slice_df = run_query(slice_query)
print("Slice Query Result")
print(slice_df)

Slice Query Result
                                game releaseDate  price
0       Borderlands Game of the Year  2023-08-31  29.99
1                  the white chamber  2023-09-07   2.99
2                      Minigame Game  2023-02-03  13.49
3                            Caliber  2023-04-12   0.00
4                            SuiCats  2023-10-19   1.99
...                              ...         ...    ...
15537          Missile Wars Playtest  2023-12-22   0.00
15538                  星域商旅 Playtest  2023-12-27   0.00
15539        Dark Age Dinos Playtest  2023-12-22   0.00
15540  The Lab and Dungeons Playtest  2023-12-26   0.00
15541        Locked In Mind Playtest  2023-12-28   0.00

[15542 rows x 3 columns]
